In [1]:
from openbabel import openbabel

In [2]:
from rdkit import Chem
from rdkit.Chem import rdBase
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import Crippen

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

In [4]:
def split(word):
    return[char for char in word]

In [5]:
def inchi(array):
    Smiles = array[0] 
    mol = Chem.MolFromSmiles(Smiles)
    dummy = Chem.inchi.MolToInchiKey(mol)#split at first layer (contains everything but stereochemical info)
    characters = split(dummy)
    dummy = ''
    position = 0
    for i in range(len(characters)):
        if characters[i] == '-':
            position = i
            break
    for i in range(position):
        dummy += characters[i]
    return(dummy)

In [6]:
def inchi_generations(array):
    Smiles = array[1]
    mol = Chem.MolFromSmiles(Smiles)
    dummy = Chem.inchi.MolToInchiKey(mol)#split at first layer (contains everything but stereochemical info)
    characters = split(dummy)
    dummy = ''
    position = 0
    for i in range(len(characters)):
        if characters[i] == '-':
            position = i
            break
    for i in range(position):
        dummy += characters[i]
    return(dummy)

In [7]:
def prep_data(library_set, test_set): #input filepaths for the 2 sets as tsv files
    library_data = pd.read_csv(library_set, sep='\t')
    test_data = pd.read_csv(test_set, sep='\t')
    print(1)
    test_data['Inchi'] = test_data.apply(inchi_generations, axis=1, raw=True, result_type='expand')
    print(2)
    #library_data['Inchi'] = library_data.apply(inchi, axis=1, raw=True, result_type='expand')
    #Using degeneracy removed datasets with Inchikey already found 
    return(library_data, test_data)

In [16]:
def find_matches(library_set, test_set):
    library_data, test_data = prep_data(library_set, test_set)
    print(3)
    test_codes = []
    for i in range(len(test_data['Inchi'])):
        test_codes.append(test_data['Inchi'][i])
    library_codes = []
    for i in range(len(library_data['Inchi'])):
        library_codes.append(library_data['Inchi'][i])
    print(4)
    matches = []
    for i in range(len(test_codes)):
        if test_codes[i] in library_codes:
            matches.append(test_codes[i])
    print(5)
    test_smiles = []
    #library_smiles = []
    matches_generations = []
    for i in range(len(matches)):
        for j in range(len(test_data['Inchi'])):
            if matches[i] == test_data['Inchi'][j]:
                test_smiles.append(test_data['Smiles'][j])
                matches_generations.append(test_data['Generation'][j])
        #for k in range(len(library_data['Inchi'])):
            #if matches[i] == library_data['Inchi'][k]:
                #library_smiles.append(library_data['Smiles'][k])
    #matches_data = {'Generation': matches_generations, 'Test Smiles': test_smiles, 'Library Smiles': library_smiles, 'Inchi': matches}
    matches_data = {'Generation': matches_generations, 'Smiles': test_smiles, 'Inchi': matches}
    library_test_matches = pd.DataFrame(matches_data)
    return(library_test_matches)

In [44]:
%%time
a = find_matches('./CHO_Data/CHO_Final.tsv', './PyruvicAcidData/pyruvic_output.tsv')

1
2
3
4
5
CPU times: user 1min 48s, sys: 701 ms, total: 1min 49s
Wall time: 1min 50s


In [45]:
a

,Generation,Smiles,Inchi


In [43]:
a.to_csv(r'NoStereoPyruvicAcid_CHNO.tsv', header=None, index=None, sep='\t', mode='a') 